In [108]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd


## Getting started and connected

In [109]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://www.petco.com/shop/en/petcostore/category/dog/dog-food/wet-dog-food')

## Closes pop-up (remove if not necessary)

In [110]:
popup_close = WebDriverWait(driver, 13).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[14]/div[3]/div/div/div/div[2]/form/div[4]/div[3]/button"))
    )
popup_close.click()

## Extracts Links and Paginates

In [111]:
prodLinks=[]
page_count = 1
while True:
    page_count += 1
    
    # extract links
    food1 = driver.find_elements_by_class_name("product-name")
    for tags in food1:
        food2 = tags.find_elements_by_tag_name("a")
        for link in food2:
            prodLinks.append(link.get_attribute("href"))
    # go through each page
    try:
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
        # another is added to keep the page 
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
    # stop if no more pages available
    except NoSuchElementException:
        break


In [112]:
# total links scraped
len(prodLinks)

708

In [113]:
# preview of some of the links in a list
prodLinks[:6]

['https://www.petco.com/shop/en/petcostore/product/wet-noses-wag-soup-turkey-and-pumpkin-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-chunky-big-texas-steak-tips-dinner-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-slow-cooked-bbq-texas-style-beef-recipe-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-grain-free-grammys-pot-pie-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/hills-science-diet-adult-perfect-weight-hearty-vegetable-and-chicken-stew-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-slow-cooked-bbq-memphis-style-chicken-recipe-wet-dog-food']

## Extracts info from each link

In [115]:
data = []
# this for loop goes through only 5 out of 708 links
for i in tqdm(prodLinks[:5]):
    driver.get(i)
    
    name = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[1]/h1').text
    price = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[3]/form/div[2]/div[1]').text

    try:
        show_more = WebDriverWait(driver, 17).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/button'))
        )
        show_more.click()
        
        
        ids = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div[1]/div/div/div[2]/ul/li[1]/p').text
        brand = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div/div/div/div[2]/ul/li[4]/p').text
        primary_flavor = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div/div/div/div[2]/ul/li[3]/p').text

    except NoSuchElementException:
        pass
    
    #collect the data
    wet_food = {'ID': ids,
                'brand': brand,
                'primary_flavor': primary_flavor,
                'name': name,
                'price': price}
    data.append(wet_food)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.43s/it]


In [116]:
data

[{'ID': '2991240',
  'brand': 'Wet Noses',
  'primary_flavor': 'Turkey',
  'name': 'Wet Noses Wag Soup Turkey and Pumpkin Wet Dog Food, 15 oz.',
  'price': '$8.99'},
 {'ID': '2302088',
  'brand': 'Merrick',
  'primary_flavor': 'Beef',
  'name': 'Merrick Chunky Grain Free Big Texas Steak Tips Dinner Wet Dog Food, 12.7 oz., Case of 12',
  'price': '$34.45\nreg $38.28'},
 {'ID': '3021237',
  'brand': 'Merrick',
  'primary_flavor': 'Beef',
  'name': 'Merrick Grain Free Slow-Cooked BBQ Texas Style with Braised Beef Wet Dog Food, 12.7 oz., Case of 12',
  'price': '$34.44\nreg $38.28'},
 {'ID': '2899291',
  'brand': 'Merrick',
  'primary_flavor': 'Chicken',
  'name': "Merrick Grain Free Grammy's Pot Pie Wet Dog Food, 12.7 oz., Case of 12",
  'price': '$34.45\nreg $38.28'},
 {'ID': '2518188',
  'brand': "Hill's",
  'primary_flavor': 'Chicken',
  'name': "Hill's Science Diet Adult Perfect Weight Hearty Vegetable & Chicken Stew Canned Dog Food, 12.5 oz., Case of 12",
  'price': '$30.99\nreg $33.

In [117]:
petco_df = pd.DataFrame(data)
petco_df

,ID,brand,primary_flavor,name,price
0,2991240,Wet Noses,Turkey,Wet Noses Wag Soup Turkey and Pumpkin Wet Dog ...,$8.99
1,2302088,Merrick,Beef,Merrick Chunky Grain Free Big Texas Steak Tips...,$34.45\nreg $38.28
2,3021237,Merrick,Beef,Merrick Grain Free Slow-Cooked BBQ Texas Style...,$34.44\nreg $38.28
3,2899291,Merrick,Chicken,Merrick Grain Free Grammy's Pot Pie Wet Dog Fo...,$34.45\nreg $38.28
4,2518188,Hill's,Chicken,Hill's Science Diet Adult Perfect Weight Heart...,$30.99\nreg $33.48


## Quick Data Cleaning

In [118]:
petco_df['price'] = petco_df['price'].apply(lambda x: x.split('\nreg'))
petco_df['new_price'] = petco_df['price'].apply(lambda x: x[0])
petco_df['reg_price'] = petco_df['price'].apply(lambda x: x[-1])
petco_df.drop('price', axis=1, inplace=True)

In [119]:
petco_df

,ID,brand,primary_flavor,name,new_price,reg_price
0,2991240,Wet Noses,Turkey,Wet Noses Wag Soup Turkey and Pumpkin Wet Dog ...,$8.99,$8.99
1,2302088,Merrick,Beef,Merrick Chunky Grain Free Big Texas Steak Tips...,$34.45,$38.28
2,3021237,Merrick,Beef,Merrick Grain Free Slow-Cooked BBQ Texas Style...,$34.44,$38.28
3,2899291,Merrick,Chicken,Merrick Grain Free Grammy's Pot Pie Wet Dog Fo...,$34.45,$38.28
4,2518188,Hill's,Chicken,Hill's Science Diet Adult Perfect Weight Heart...,$30.99,$33.48
